In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from PIL import Image

In [2]:

import matplotlib.pyplot as plt
%matplotlib inline
import skimage.io as io

In [3]:
labels = pd.read_csv("D:\RajatLearning\DataScience\kaggle\dogbreed\labels.csv")
labels.head()

,id,breed
0,000bec180eb18c7604dcecc8fe0dba07,boston_bull
1,001513dfcb2ffafc82cccf4d8bbaba97,dingo
2,001cdf01b096e06d78e9e5112d419397,pekinese
3,00214f311d5d2247d5dfe4fe24b2303d,bluetick
4,0021f9ceb3235effd7fcde7f7538ed62,golden_retriever


In [9]:
unique_breeds = labels.breed.unique()
len(unique_breeds)

120

In [10]:
gr_labels = labels.groupby("breed").count()
gr_labels = gr_labels.rename(columns = {"id" : "count"})
gr_labels = gr_labels.sort_values("count", ascending=False)
gr_labels.head()

,count
breed,
scottish_deerhound,126
maltese_dog,117
afghan_hound,116
entlebucher,115
bernese_mountain_dog,114


In [ ]:
scottish_deerhound_id = labels.loc[labels.breed == "scottish_deerhound"].iloc[0, 0]
Image.open("D:\RajatLearning\DataScience\kaggle\dogbreed\dogtrain\\"+scottish_deerhound_id+".jpg")

In [19]:
gr_labels.tail()

,count
breed,
golden_retriever,67
brabancon_griffon,67
komondor,67
briard,66
eskimo_dog,66


In [ ]:
eskimo_dog_id = labels.loc[labels.breed == "eskimo_dog"].iloc[1, 0] #0 row is too agressive, so i decided to take 1st :)
Image.open("D:\RajatLearning\DataScience\kaggle\dogbreed\dogtrain\\"+eskimo_dog_id+".jpg")

In [ ]:
from skimage.color import rgb2gray
img_grayscale = rgb2gray(img)
plt.imshow(img_grayscale)

# new code##

In [ ]:
scottish_deerhound_id = labels.loc[labels.breed == "scottish_deerhound"].iloc[1, 0]
img=Image.open("D:\RajatLearning\DataScience\kaggle\dogbreed\dogtrain\\"+scottish_deerhound_id+".jpg")
img=img.resize((200,200),Image.ANTIALIAS)
bimg=np.array(img.getdata())
bimg=bimg.flatten()
labels[scottish_deerhound_id]
#img=io.imread("D:\RajatLearning\DataScience\kaggle\dogbreed\dogtrain\\"+scottish_deerhound_id+".jpg")

In [16]:
item =labels.iloc[0:,0][2]
sd=io.imread("D:\RajatLearning\DataScience\kaggle\dogbreed\dogtrain\\"+item+".jpg")
#img=sd.resize((200,200),Image.ANTIALIAS)


In [28]:
bimg=[]
for 
item =labels.iloc[0:,0][2]
img=Image.open("D:\RajatLearning\DataScience\kaggle\dogbreed\dogtrain\\"+item+".jpg")
img=img.resize((200,200),Image.ANTIALIAS)
bimg=np.array(img.getdata())
bimg.append(bimg.flatten())


In [6]:
labels["ImgData"]= labels['id'].apply(getImage)

In [4]:
def getImage(id):
    
    #item =labels.iloc[0:,0][2]
    img=Image.open("D:\RajatLearning\DataScience\kaggle\dogbreed\dogtrain\\"+id+".jpg")
    img=img.resize((200,200),Image.ANTIALIAS)
    bimg=np.array(img.getdata())/255
    return bimg.flatten()

In [41]:
labels.head()

,id,breed,ImgData
0,000bec180eb18c7604dcecc8fe0dba07,boston_bull,"[0.627450980392, 0.392156862745, 0.2, 0.627450..."
1,001513dfcb2ffafc82cccf4d8bbaba97,dingo,"[0.23137254902, 0.164705882353, 0.137254901961..."
2,001cdf01b096e06d78e9e5112d419397,pekinese,"[0.4, 0.396078431373, 0.376470588235, 0.447058..."
3,00214f311d5d2247d5dfe4fe24b2303d,bluetick,"[0.309803921569, 0.294117647059, 0.29019607843..."
4,0021f9ceb3235effd7fcde7f7538ed62,golden_retriever,"[0.325490196078, 0.333333333333, 0.32156862745..."


In [ ]:
for item in labels["id"]:
    img=Image.open("D:\RajatLearning\DataScience\kaggle\dogbreed\dogtrain\\"+item+".jpg")
    img=img.resize((200,200),Image.ANTIALIAS)
    bimg=np.array(img.getdata())
    bimg.append(bimg.flatten())

In [150]:
item =labels.iloc[0:,0][2]
sd=Image.open("D:\RajatLearning\DataScience\kaggle\dogbreed\dogtrain\\"+item+".jpg")


In [5]:
labels.head()

,id,breed
0,000bec180eb18c7604dcecc8fe0dba07,boston_bull
1,001513dfcb2ffafc82cccf4d8bbaba97,dingo
2,001cdf01b096e06d78e9e5112d419397,pekinese
3,00214f311d5d2247d5dfe4fe24b2303d,bluetick
4,0021f9ceb3235effd7fcde7f7538ed62,golden_retriever


In [169]:
names=pd.DataFrame(labels.iloc[0:,0])
#names=list(labels.iloc[0:,0])

In [160]:
names['Index']=names.index

In [ ]:
names.set_index("Index")

In [148]:
names['Image']=np.nan

In [155]:
names.head()

,id
0,000bec180eb18c7604dcecc8fe0dba07
1,001513dfcb2ffafc82cccf4d8bbaba97
2,001cdf01b096e06d78e9e5112d419397
3,00214f311d5d2247d5dfe4fe24b2303d
4,0021f9ceb3235effd7fcde7f7538ed62


In [167]:
type(names)

list

# Recignition

In [20]:
from sklearn.model_selection import train_test_split

In [7]:
dataX=labels['ImgData']

In [8]:
dataX.head()

0    [0.627450980392, 0.392156862745, 0.2, 0.627450...
1    [0.23137254902, 0.164705882353, 0.137254901961...
2    [0.4, 0.396078431373, 0.376470588235, 0.447058...
3    [0.309803921569, 0.294117647059, 0.29019607843...
4    [0.325490196078, 0.333333333333, 0.32156862745...
Name: ImgData, dtype: object

In [18]:
dataY=labels['breed']

In [21]:
X_train, X_test, y_train, y_test = train_test_split(dataX, dataY, test_size=0.33, random_state=42)

In [22]:
from sklearn.svm import SVC

In [23]:
est=SVC()

In [24]:
est.fit(X_train,y_train)

ValueError: setting an array element with a sequence.

In [ ]:
espredict=est.predict(X_test)

In [ ]:
from sklearn.metrics import confusion_matrix,classification_report

In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
rfc500=RandomForestClassifier(n_estimators=500,verbose=3)

In [ ]:
rfc500.fit(X_train,y_train)

In [ ]:
predictRF500=rfc500.predict(X_test)

# roughwork

In [ ]:
td= labels['id'].head().apply(getImage)

In [ ]:
def getImage(id):
    
    #item =labels.iloc[0:,0][2]
    img=Image.open("D:\RajatLearning\DataScience\kaggle\dogbreed\dogtrain\\"+id+".jpg")
    img=img.resize((200,200),Image.ANTIALIAS)
    bimg=np.array(img.getdata())/255
    return bimg.flatten()

In [ ]:
#000bec180eb18c7604dcecc8fe0dba07

In [13]:
id='000bec180eb18c7604dcecc8fe0dba07'
img=Image.open("D:\RajatLearning\DataScience\kaggle\dogbreed\dogtrain\\"+id+".jpg")
img=img.resize((200,200),Image.ANTIALIAS)
bimg=np.array(img.getdata())/255
bimg=bimg.flatten()

array([ 0.62745098,  0.39215686,  0.2       , ...,  0.76470588,
        0.56470588,  0.3372549 ])